In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Stan関連
import cmdstanpy
from cmdstanpy import CmdStanModel

# Global PPM モジュールのインポート
from nm.research.global_ppm.py.data_manager import FactSetDataManager
from nm.research.global_ppm.py.bayesian_data_processor import BayesianDataProcessor
from nm.research.global_ppm.py.bayesian_model_builder import BayesianModelBuilder
from nm.research.global_ppm.py.bayesian_inference import BayesianInference

# 日本語表示対応
import japanize_matplotlib
japanize_matplotlib.japanize()

# 可視化設定
plt.style.use('fivethirtyeight')
sns.set_palette("husl")
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

print("✅ ライブラリのインポート完了")


In [ ]:
# 設定
DATA_PROCESSOR_PATH = "./bayesian_data/processed_data.pkl"  # BayesianDataProcessorで作成されたデータ
OUTPUT_DIR = Path("./bayesian_results")
OUTPUT_DIR.mkdir(exist_ok=True)

# 変分推論パラメータ
VARIATIONAL_PARAMS = {
    "draws": 2000,
    "iter": 500000,
    "seed": 1234,
    "show_console": True,
    "require_converged": False
}

# モデル設定
MODEL_NAME = "hierarchical_roic"

print(f"📁 データパス: {DATA_PROCESSOR_PATH}")
print(f"📁 出力ディレクトリ: {OUTPUT_DIR}")
print(f"🔧 変分推論パラメータ: {VARIATIONAL_PARAMS}")

# データ存在チェック
if not Path(DATA_PROCESSOR_PATH).exists():
    print(f"⚠️  警告: データファイルが見つかりません: {DATA_PROCESSOR_PATH}")
    print("先にfactset_bayesian_dataset_creation.ipynbを実行してデータを作成してください。")
else:
    print("✅ データファイルが確認できました。")


In [ ]:
# 処理済みデータの読み込み
print("📖 データセットを読み込み中...")

with open(DATA_PROCESSOR_PATH, 'rb') as f:
    processed_data = pickle.load(f)

# データ内容の確認
stan_data = processed_data['stan_data']
product_names = processed_data['product_names']
processing_info = processed_data['processing_info']

print(f"✅ データ読み込み完了")
print(f"📊 データセット情報:")
print(f"  - 期間: {processing_info['start_period']} - {processing_info['end_period']}")
print(f"  - 製品数: {processing_info['n_products']}")
print(f"  - セグメント数: {processing_info['n_segments']}")
print(f"  - 企業数: {processing_info['n_companies']}")
print(f"  - 時間期間数: {processing_info['n_time_periods']}")
print(f"  - セグメント観測数: {stan_data['N_obs']}")
print(f"  - 連結観測数: {stan_data['N_obs_consol']}")

print(f"\n🏷️  製品カテゴリ例 (最初の10個):")
for i, name in enumerate(product_names[:10]):
    print(f"  {i+1}. {name}")


In [ ]:
# Stanモデルビルダーの初期化
print("🔨 Stanモデルを準備中...")
model_builder = BayesianModelBuilder(model_dir=str(OUTPUT_DIR))

# 階層ベイジアンROICモデルの作成・コンパイル
model = model_builder.create_hierarchical_roic_model(MODEL_NAME, force_compile=True)
print("✅ Stanモデルのコンパイル完了")

# データをJSONファイルとして保存
data_json_path = OUTPUT_DIR / "stan_data.json"
model_builder.save_data_json(stan_data, data_json_path)
print(f"💾 Stan用データを保存: {data_json_path}")

# 推論エンジンの初期化
print("🚀 変分推論を開始...")
inference = BayesianInference(output_dir=OUTPUT_DIR)

# 変分推論の実行
fit = inference.run_variational_inference(
    model=model,
    data=str(data_json_path),
    **VARIATIONAL_PARAMS
)

print("🎉 変分推論完了！")
print(f"推論結果タイプ: {type(fit)}")

# 結果の保存
inference_result_path = OUTPUT_DIR / f"inference_result_{MODEL_NAME}.pkl"
with open(inference_result_path, 'wb') as f:
    pickle.dump(fit, f)
print(f"💾 推論結果を保存: {inference_result_path}")


In [ ]:
# 推論結果からパラメータを抽出
print("📊 推論結果を分析中...")

# パラメータの抽出（Item_ROICが主要パラメータ）
try:
    # 変分推論結果からサンプルを取得
    samples = fit.variational_sample
    item_roic_samples = samples["Item_ROIC"]  # [draws, Product_N, Time_N]
    
    print(f"✅ パラメータ抽出完了")
    print(f"Item_ROIC形状: {item_roic_samples.shape}")
    print(f"  - サンプル数: {item_roic_samples.shape[0]}")
    print(f"  - 製品数: {item_roic_samples.shape[1]}")
    print(f"  - 時間期間数: {item_roic_samples.shape[2]}")
    
    # 統計量の計算
    item_roic_mean = np.mean(item_roic_samples, axis=0)  # [Product_N, Time_N]
    item_roic_std = np.std(item_roic_samples, axis=0)
    item_roic_q025 = np.percentile(item_roic_samples, 2.5, axis=0)
    item_roic_q975 = np.percentile(item_roic_samples, 97.5, axis=0)
    
    print(f"\n📈 製品ROIC統計（最新期間）:")
    latest_mean = item_roic_mean[:, -1]
    print(f"  - 平均ROIC: {np.mean(latest_mean):.4f}")
    print(f"  - 標準偏差: {np.std(latest_mean):.4f}")
    print(f"  - 最小値: {np.min(latest_mean):.4f}")
    print(f"  - 最大値: {np.max(latest_mean):.4f}")
    
    # トップ・ボトム製品の特定
    latest_rank = np.argsort(latest_mean)
    top_5_indices = latest_rank[-5:]
    bottom_5_indices = latest_rank[:5]
    
    print(f"\n🏆 上位5製品（最新期間ROIC）:")
    for i, idx in enumerate(reversed(top_5_indices)):
        print(f"  {i+1}. {product_names[idx]}: {latest_mean[idx]:.4f}")
    
    print(f"\n⬇️  下位5製品（最新期間ROIC）:")
    for i, idx in enumerate(bottom_5_indices):
        print(f"  {i+1}. {product_names[idx]}: {latest_mean[idx]:.4f}")
        
except Exception as e:
    print(f"❌ パラメータ抽出でエラー: {e}")
    raise


In [ ]:
# 製品ROIC可視化
print("📊 可視化を作成中...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('製品ROIC分析結果', fontsize=18, fontweight='bold')

# 1. 全製品平均ROICの時系列
avg_roic_over_time = np.mean(item_roic_mean, axis=0)
axes[0, 0].plot(avg_roic_over_time, linewidth=3, color='darkblue', marker='o')
axes[0, 0].set_title('全製品平均ROIC時系列', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('期間')
axes[0, 0].set_ylabel('平均ROIC')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# 2. 最新期間の製品ROIC分布
axes[0, 1].hist(latest_mean, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 1].axvline(np.mean(latest_mean), color='red', linestyle='--', linewidth=2, label=f'平均: {np.mean(latest_mean):.3f}')
axes[0, 1].set_title('最新期間の製品ROIC分布', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('ROIC')
axes[0, 1].set_ylabel('製品数')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 上位・下位製品の時系列比較
colors_top = plt.cm.Reds(np.linspace(0.5, 1.0, 5))
colors_bottom = plt.cm.Blues(np.linspace(0.5, 1.0, 5))

for i, idx in enumerate(reversed(top_5_indices)):
    axes[1, 0].plot(item_roic_mean[idx, :], 
                   label=f'{product_names[idx][:20]}...', 
                   color=colors_top[i], linewidth=2)

for i, idx in enumerate(bottom_5_indices):
    axes[1, 0].plot(item_roic_mean[idx, :], 
                   label=f'{product_names[idx][:20]}...', 
                   color=colors_bottom[i], linewidth=2, linestyle='--')

axes[1, 0].set_title('上位・下位製品ROIC時系列比較', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('期間')
axes[1, 0].set_ylabel('ROIC')
axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)

# 4. ROIC標準偏差（不確実性）の時系列
roic_uncertainty = np.mean(item_roic_std, axis=0)
axes[1, 1].plot(roic_uncertainty, linewidth=3, color='green', marker='s')
axes[1, 1].set_title('製品ROIC不確実性（標準偏差）時系列', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('期間')
axes[1, 1].set_ylabel('平均標準偏差')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'product_roic_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 可視化完了")


In [ ]:
# 結果のエクスポート
print("💾 結果をエクスポート中...")

# 1. 製品ROIC結果をDataFrameとして保存
product_roic_df = pd.DataFrame(
    item_roic_mean.T,  # 転置して時間×製品の形に
    columns=product_names
)
product_roic_df.index.name = 'Period'
product_roic_df.to_csv(OUTPUT_DIR / 'product_roic_timeseries.csv')

# 2. 最新期間の製品ランキング
latest_ranking_df = pd.DataFrame({
    'Product': product_names,
    'ROIC_Mean': latest_mean,
    'ROIC_Std': item_roic_std[:, -1],
    'ROIC_Q025': item_roic_q025[:, -1],
    'ROIC_Q975': item_roic_q975[:, -1]
}).sort_values('ROIC_Mean', ascending=False)
latest_ranking_df['Rank'] = range(1, len(latest_ranking_df) + 1)
latest_ranking_df.to_csv(OUTPUT_DIR / 'product_roic_ranking_latest.csv', index=False)

# 3. 統計サマリーの保存
summary_stats = {
    'dataset_info': processing_info,
    'inference_params': VARIATIONAL_PARAMS,
    'model_name': MODEL_NAME,
    'results_summary': {
        'total_products': len(product_names),
        'latest_period_avg_roic': float(np.mean(latest_mean)),
        'latest_period_std_roic': float(np.std(latest_mean)),
        'latest_period_min_roic': float(np.min(latest_mean)),
        'latest_period_max_roic': float(np.max(latest_mean)),
        'top_5_products': [product_names[idx] for idx in reversed(top_5_indices)],
        'bottom_5_products': [product_names[idx] for idx in bottom_5_indices],
        'inference_timestamp': datetime.now().isoformat()
    }
}

with open(OUTPUT_DIR / 'modeling_summary.json', 'w', encoding='utf-8') as f:
    import json
    json.dump(summary_stats, f, ensure_ascii=False, indent=2)

print("✅ エクスポート完了")
print(f"\n📁 保存されたファイル:")
print(f"  - product_roic_timeseries.csv: 製品ROIC時系列データ")
print(f"  - product_roic_ranking_latest.csv: 最新期間製品ランキング")
print(f"  - product_roic_analysis.png: 分析グラフ")
print(f"  - modeling_summary.json: モデリング全体サマリー")
print(f"  - inference_result_{MODEL_NAME}.pkl: 推論結果（Pythonオブジェクト）")
print(f"  - stan_data.json: Stan入力データ")

# 成功サマリーの表示
print(f"\n" + "="*60)
print("🎉 FactSet ベイジアン変分推論完了!")
print("="*60)
print(f"📊 分析結果サマリー:")
print(f"  期間: {processing_info['start_period']} - {processing_info['end_period']}")
print(f"  製品数: {processing_info['n_products']}")
print(f"  最新期間平均ROIC: {np.mean(latest_mean):.4f}")
print(f"  最高製品ROIC: {np.max(latest_mean):.4f} ({product_names[latest_rank[-1]]})")
print(f"  最低製品ROIC: {np.min(latest_mean):.4f} ({product_names[latest_rank[0]]})")

print(f"\n💡 活用方法:")
print(f"  - 製品ポートフォリオ戦略の策定")
print(f"  - 高収益製品カテゴリへの投資集中")
print(f"  - 低収益製品の改善・撤退検討")
print(f"  - 時系列パターンによる将来予測")
print(f"  - 企業評価・投資判断への活用")
